In [109]:
import pandas as pd
import numpy as np
import tweepy
import jsonpickle

# Chaves para consumir tweets
CONSUMER_KEY    = '2vyrYQPStDif9GqueM16HSYDA'
CONSUMER_SECRET = 'ztECj8y4kKulTIawI7yQsVLP3suc1WJ8bMWiRWrAfXeWnworF9'

# Acesso para o tweeter
ACCESS_TOKEN  = '3232348367-gV4ezQHeIZFbFL1JB699IhZf7RifMIWGr5nAhnG'
ACCESS_SECRET = 'LuIbWgoKBdizBbuioeT1jy9NCzshznxa3djE056fEGkF4'

# Configuração de acesso
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)    
    api = tweepy.API(auth)
    return api
 
api = connect_to_twitter_OAuth()

In [110]:
def get_save_tweets(filepath, api, query, max_tweets=30000, lang='pt', delta_days = None):

    tweetCount = 0
    
    with open(filepath, 'w') as f:
        
        if delta_days is None:
            tweets = tweepy.Cursor(api.search,q=query,lang=lang).items(max_tweets)
        else :
            import datetime        
            now = datetime.date.today()
            since = now - datetime.timedelta(delta_days)         
            tweets = tweepy.Cursor(api.search,q=query,lang=lang,since=since,until=now).items(max_tweets)
                
        for tweet in tweets:            
            # Convert Tweet to JSON format
            tweet_encoded = jsonpickle.encode(tweet._json)
            f.write( tweet_encoded + '\n')
            tweetCount += 1
        
        print("Downloaded {0} tweets".format(tweetCount))                

In [111]:
query = '#lages'

# Get those tweets
get_save_tweets('tweets.json', api, query, delta_days = 100)

Downloaded 103 tweets


In [112]:
def tweets_to_df(path):
    
    tweets = list(open(path, 'rt'))
    
    text = []
    weekday = []
    month = []
    day = []
    hour = []
    hashtag = []
    url = []
    favorite = []
    reply = []
    retweet = []
    follower = []
    following = []
    user = []
    screen_name = []

    for t in tweets:
        t = jsonpickle.decode(t)
        
        # Text
        text.append(t['text'])
        
        # Decompose date
        date = t['created_at']
        weekday.append(date.split(' ')[0])
        month.append(date.split(' ')[1])
        day.append(date.split(' ')[2])
        
        time = date.split(' ')[3].split(':')
        hour.append(time[0]) 
        
        # Has hashtag
        if len(t['entities']['hashtags']) == 0:
            hashtag.append(0)
        else:
            hashtag.append(1)
            
        # Has url
        if len(t['entities']['urls']) == 0:
            url.append(0)
        else:
            url.append(1)
            
        # Number of favs
        favorite.append(t['favorite_count'])
        
        # Is reply?
        if t['in_reply_to_status_id'] == None:
            reply.append(0)
        else:
            reply.append(1)       
        
        # Retweets count
        retweet.append(t['retweet_count'])
        
        # Followers number
        follower.append(t['user']['followers_count'])
        
        # Following number
        following.append(t['user']['friends_count'])
        
        # Add user
        user.append(t['user']['name'])

        # Add screen name
        screen_name.append(t['user']['screen_name'])
        
    d = {'text': text,
         'weekday': weekday,
         'month' : month,
         'day': day,
         'hour' : hour,
         'has_hashtag': hashtag,
         'has_url': url,
         'fav_count': favorite,
         'is_reply': reply,
         'retweet_count': retweet,
         'followers': follower,
         'following' : following,
         'user': user,
         'screen_name' : screen_name
        }
    
    return pd.DataFrame(data = d)
        
tweets_df = tweets_to_df('tweets.json')

In [113]:
tweets_df

,text,weekday,month,day,hour,has_hashtag,has_url,fav_count,is_reply,retweet_count,followers,following,user,screen_name
0,RT @cbmscituporanga: 🚑 🚒\n.\nCapotamento de ve...,Mon,Apr,15,22,1,0,0,0,1,71,164,Luis Fernando,lnandofelipe
1,🚑 🚒\n.\nCapotamento de veículo atendido hoje p...,Mon,Apr,15,22,1,1,1,0,1,48,169,Corpo de Bombeiros Militar de Ituporanga,cbmscituporanga
2,"17:34 #pré_hospitalar local: saldanha amaral, ...",Mon,Apr,15,20,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
3,"16:06 #pré_hospitalar local: luis de camões, C...",Mon,Apr,15,19,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
4,"15:41 #pré_hospitalar local: 31 de março, São ...",Mon,Apr,15,18,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
5,14:42 #pré_hospitalar local: Avenida Segundo B...,Mon,Apr,15,17,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
6,10:34 #auxílios_apoios local: Rua Bernardo Gon...,Mon,Apr,15,13,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
7,Está chegando o dia!\nPrinting Day - Lages.\n\...,Mon,Apr,15,12,0,1,1,0,0,356,437,HELIOPRINT,HELIOPRINT
8,08:02 #pré_hospitalar local: Rua Saturnino Ant...,Mon,Apr,15,11,1,0,0,0,0,8525,27,CBMSC193,CBMSC193
9,"04:59 #pré_hospitalar local: Rua Tito Ramos, U...",Mon,Apr,15,07,1,0,0,0,0,8525,27,CBMSC193,CBMSC193


In [114]:
for i in range(10):
    print(tweets_df.text[i])

RT @cbmscituporanga: 🚑 🚒
.
Capotamento de veículo atendido hoje pela manhã em Ituporanga.
.
.
#cbmsc #ituporanga #bombeiros #santacatarina…
🚑 🚒
.
Capotamento de veículo atendido hoje pela manhã em Ituporanga.
.
.
#cbmsc #ituporanga #bombeiros… https://t.co/e77tHaJDoU
17:34 #pré_hospitalar local: saldanha amaral, Universitario, #Lages - queda de motociclista por atropelar cachorro.
16:06 #pré_hospitalar local: luis de camões, Coral, #Lages - ferimento na cabeça de masculino maior que cai em escada igreja.
15:41 #pré_hospitalar local: 31 de março, São Vicente, #Lages - queda de bicicleta de masculino menor.
14:42 #pré_hospitalar local: Avenida Segundo Batalhão Rodoviário, Conta Dinheiro, #Lages - queda de telhado de masculino maior.
10:34 #auxílios_apoios local: Rua Bernardo Gonçalves Kuster, 0, #Lages - apoio a usa ( samu ) , choque elétrico  em masculino
Está chegando o dia!
Printing Day - Lages.

Ótima oportunidade para conhecer de perto os principais equipamentos di… https://t.co/lvW

In [115]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
for i in range(10):
    print(tokenizer.tokenize( tweets_df.text[i]) )

['RT', '@cbmscituporanga', ':', '🚑', '🚒', '.', 'Capotamento', 'de', 'veículo', 'atendido', 'hoje', 'pela', 'manhã', 'em', 'Ituporanga', '.\n.\n.', '#cbmsc', '#ituporanga', '#bombeiros', '#santacatarina', '…']
['🚑', '🚒', '.', 'Capotamento', 'de', 'veículo', 'atendido', 'hoje', 'pela', 'manhã', 'em', 'Ituporanga', '.\n.\n.', '#cbmsc', '#ituporanga', '#bombeiros', '…', 'https://t.co/e77tHaJDoU']
['17:34', '#pré_hospitalar', 'local', ':', 'saldanha', 'amaral', ',', 'Universitario', ',', '#Lages', '-', 'queda', 'de', 'motociclista', 'por', 'atropelar', 'cachorro', '.']
['16:06', '#pré_hospitalar', 'local', ':', 'luis', 'de', 'camões', ',', 'Coral', ',', '#Lages', '-', 'ferimento', 'na', 'cabeça', 'de', 'masculino', 'maior', 'que', 'cai', 'em', 'escada', 'igreja', '.']
['15:41', '#pré_hospitalar', 'local', ':', '31', 'de', 'março', ',', 'São', 'Vicente', ',', '#Lages', '-', 'queda', 'de', 'bicicleta', 'de', 'masculino', 'menor', '.']
['14:42', '#pré_hospitalar', 'local', ':', 'Avenida', 'Seg